In [23]:
import pandas as pd 
import numpy as np
import requests
import json

from config import API_KEY

In [24]:
minha_chave = API_KEY

# para conseguir acessar a API, precisamos incluir no cabeçalho da requisição a nossa chave, que definimos anteriormente. Não é necessário alterar essa variável.
chave = {'chave-api-dados':API_KEY} 

In [25]:
url = f'http://www.transparencia.gov.br/api-de-dados/despesas/por-orgao?ano=2020&orgaoSuperior=154040&pagina=1'
# esta URL foi gerada a partir da consulta por órgão no swagger
   
# aqui, fazemos a consulta e obtemos o resultado para cada ano em JSON
r_orgao = requests.get(url, verify=True,headers=chave).json()

In [27]:
r_orgao

[]